![](https://algo-assets.amplifyme.com/quant/challenge1.png)

# Quant Simulation Challenge 1 - Non-skewed Price Making

Your first challenge is to work independently to follow the instructor to complete the challenges below. This will help give you the foundations for the future challenges.

The goal of this challenge is to automate the market-making process, using object-orientated programming to automatically create a bid-offer spread around the reference price. You're expected to provide a 2% non-skewed bid and offer for each trade.

Good luck!

To start let's import the packages to be used within this notebook.

In [63]:
# The code in this cell is used to import the packages to be used throughout this notebook.
# The following are private packages available only during this simnulation:
from AmplifyQuantTrading import Data
from AmplifyQuantTrading import Exchange
from AmplifyQuantTrading import MarketMaker
from AmplifyQuantTrading import HedgeFund as hf
# The following are publicly available packages:
from matplotlib import pyplot as plt
from pandas import *
from IPython.display import VimeoVideo

In [64]:
VimeoVideo('744733270?h=2c04396a12')

In [65]:
# HIDDEN PARAMS BOX

The code below will assign the prices data series and price_requests data series to the two variables to make them available throughout the project.

In [66]:
prices = Data.get_price_series("PricestoFeedserver")
price_requests = Data.get_price_requests("PriceRequeststoFeedserver")

### a) Iterate through the first ten price requests and append to the test_requests list.

The price requests are being reduced to just 10 items to allow for easier creation of an algorithm on a smaller dataset.

In [67]:
VimeoVideo('744733424?h=f4a831df4d')

In [68]:
test_requests = []

In [69]:
for index in range(0, 10):  # Select the first ten prices in the data.
    price = prices[index]
    request = price_requests[index]

    print(f"Index: {index}, Price: {price}, Request: {request}")

Index: 0, Price: ['AAPL', 0, 74.7], Request: ['FB', 0, 3010]
Index: 1, Price: ['FAANG', 0, 117.0], Request: ['AAPL', 10000, 6929]
Index: 2, Price: ['FB', 0, 194.44], Request: ['FB', 30000, 7015]
Index: 3, Price: ['AAPL', 5000, 74.81], Request: ['FB', 70000, 4910]
Index: 4, Price: ['FAANG', 5000, 117.27], Request: ['AAPL', 105000, 5783]
Index: 5, Price: ['FB', 5000, 193.53], Request: ['AAPL', 110000, 3280]
Index: 6, Price: ['AAPL', 10000, 74.63], Request: ['FB', 115000, 2465]
Index: 7, Price: ['FAANG', 10000, 120.36], Request: ['AAPL', 120000, 656]
Index: 8, Price: ['FB', 10000, 195.91], Request: ['FB', 130000, 1948]
Index: 9, Price: ['AAPL', 15000, 73.83], Request: ['FB', 150000, 1386]


Now that you have appended the items to **test_requests** you can find them in the output below.

The output should follow the format **[ [ ticker, date, volume ], [ ticker, date, volume ], ... ]**

In [70]:
print(test_requests)

[]


##### The cell below is used to grade the work after the event. You do not need to do anything here.

In [71]:
# GRADING CELL

### b) Identify the reference prices for the first ten requests

Now iterate through the *test_requests* and match them with the relevant *date* and *ticker* in the *prices* variable. Store these reference prices and price_requests in a list called *request_with_prices*.

#### request_with_prices should be in the format [ ([price requests], ref_price), ([price requests], ref_price), ...  ]

In [72]:
VimeoVideo('744733584?h=7b0768fc51')

In [73]:
request_with_prices = []

In [74]:
import numpy as np

def create_request_with_prices(prices, price_requests):
    """
    Matches all price requests with their corresponding reference prices.

    Args:
        prices: A list of lists representing prices, with each inner list 
                in the format [ticker, date/index, price].
        price_requests: A list of lists representing price requests, with each 
                       inner list in the format [ticker, date/index, volume].

    Returns:
        A list of tuples, where each tuple contains a price request and its 
        corresponding reference price: 
        [([price request], ref_price), ([price request], ref_price), ...].
    """

    # Convert to NumPy arrays for efficient operations
    prices_arr = np.array(prices, dtype=object)
    price_requests_arr = np.array(price_requests, dtype=object)

    # Match requests with prices and create request_with_prices
    request_with_prices = []

    for request in price_requests_arr:  # Iterate through all price requests
        ticker, req_index, _ = request  # Extract ticker and request index

        # Find matching price using boolean indexing
        matching_price_index = np.where((prices_arr[:, 0] == ticker) & (prices_arr[:, 1] == req_index))[0]

        if matching_price_index.size > 0:
            ref_price = prices_arr[matching_price_index[0], 2]
            request_with_prices.append((request, ref_price))

    return request_with_prices

In [75]:
request_with_prices = create_request_with_prices(prices,price_requests)[:10]

Run the cell below to check the output for test_requests.

##### The output from test_requests should be in the format: [ ( [price requests], ref_price ), ( [price requests], ref_price ), ... ]

In [76]:
print(request_with_prices)

[(array(['FB', 0, 3010], dtype=object), 194.44), (array(['AAPL', 10000, 6929], dtype=object), 74.63), (array(['FB', 30000, 7015], dtype=object), 195.75), (array(['FB', 70000, 4910], dtype=object), 193.3), (array(['AAPL', 105000, 5783], dtype=object), 73.13), (array(['AAPL', 110000, 3280], dtype=object), 73.83), (array(['FB', 115000, 2465], dtype=object), 185.76), (array(['AAPL', 120000, 656], dtype=object), 76.56), (array(['FB', 130000, 1948], dtype=object), 191.32), (array(['FB', 150000, 1386], dtype=object), 195.82)]



##### The cell below is used to grade the work after the event. You do not need to do anything here.

In [77]:
# GRADING CELL

### c) Create a non-skewed bid and offer for the request_with_prices

Now that you have paired the Request with a Reference Price you can create your bid and offer. We will be using Object Oriented Programming here to pass our trade to the Hedge Fund later on.

### **QuotedTrade object**
This object will be used to quote trades and send them to the hedge funds. The object contains the following attributes:
* ticker: String
* trade_volume: Integer
* ref_price: Float
* bid_price: Float
* offer_price: Float
* date: Integer

In [78]:
VimeoVideo('744733708?h=0973a3bbe4')

In [79]:
class QuotedTrade:
    def __init__(self, ticker, trade_volume, ref_price, bid_price, offer_price,
                 date):
        self.ticker = ticker
        self.trade_volume = trade_volume
        self.ref_price = ref_price
        self.bid_price = bid_price
        self.offer_price = offer_price
        self.date = date

    def __str__(self):
        return f'Trade Request for {self.ticker}, {self.trade_volume} shares @ {self.ref_price} on {self.date}. Bid Price: {self.bid_price} and Offer Price: {self.offer_price}'

    def __repr__(self):
        return f'QuotedTrade(ticker={self.ticker}, trade_volume={self.trade_volume}, ref_price={self.ref_price}, bid_price={self.bid_price}, offer_price={self.offer_price}, date={self.date})'

Now that you have initialised the QuotedTrade object you can use this as a template for trades.

To do this create your bid and offer in the task below by assigning a 2% spread on each side based on the reference price.

In [80]:
quoted_trades = []

In [81]:

def create_quoted_trades(request_with_prices):
    """
    Creates QuotedTrade objects with a 2% non-skewed spread.

    Args:
        request_with_prices: A list of tuples in the format 
                             [([price request], ref_price), ...].

    Returns:
        A list of QuotedTrade objects.
    """
    quoted_trades = []
    for request, ref_price in request_with_prices:
        ticker, date, trade_volume = request  # Extract info from the request array
        
        spread = 0.02  # 2% spread
        bid_price = ref_price * (1 - spread / 2)
        offer_price = ref_price * (1 + spread / 2)

        quoted_trade = QuotedTrade(ticker, trade_volume, ref_price, bid_price, offer_price, date)
        quoted_trades.append(quoted_trade)
    return quoted_trades

In [82]:
quoted_trades = create_quoted_trades(request_with_prices)[:10]

Run the cell below to check the output for quoted_trades. 

##### In the output below you should see many QuotedTrade objects with their details.

In [83]:
print(quoted_trades)

[QuotedTrade(ticker=FB, trade_volume=3010, ref_price=194.44, bid_price=192.4956, offer_price=196.3844, date=0), QuotedTrade(ticker=AAPL, trade_volume=6929, ref_price=74.63, bid_price=73.88369999999999, offer_price=75.3763, date=10000), QuotedTrade(ticker=FB, trade_volume=7015, ref_price=195.75, bid_price=193.7925, offer_price=197.7075, date=30000), QuotedTrade(ticker=FB, trade_volume=4910, ref_price=193.3, bid_price=191.36700000000002, offer_price=195.233, date=70000), QuotedTrade(ticker=AAPL, trade_volume=5783, ref_price=73.13, bid_price=72.39869999999999, offer_price=73.8613, date=105000), QuotedTrade(ticker=AAPL, trade_volume=3280, ref_price=73.83, bid_price=73.0917, offer_price=74.5683, date=110000), QuotedTrade(ticker=FB, trade_volume=2465, ref_price=185.76, bid_price=183.9024, offer_price=187.61759999999998, date=115000), QuotedTrade(ticker=AAPL, trade_volume=656, ref_price=76.56, bid_price=75.7944, offer_price=77.32560000000001, date=120000), QuotedTrade(ticker=FB, trade_volume=

##### The cell below is used to grade the work after the event. You do not need to do anything here.

In [84]:
# GRADING CELL

### d) Interact with the Hedge Fund to Show quoted_trades and recieve a response.

Using the *quoted_trades* list interact with the HF object to receive a "Buy, Sell, Refuse" response from the HF and store these *HfResponse* objects in an list called *hf_responses*.

##### Sends the quoted_trade object to the hedge fund to make a decision for a trade.
```python
hf.show(QuotedTrade)
```
***Parameters:***
* QuotedTrade: Custom Object

***Returns:***
* HfResponse( ticker: String, trade_volume: Integer, trade_price: Float, hf_action: String, ref_price: Float, bid_price: Float, offer_price: Float, date: Integer )

In [85]:
VimeoVideo('744734042?h=54b48fb5bb')

In [86]:
hf_responses = []

In [87]:
def get_hf_responses(quoted_trades):
    hf_responses = []
    for trade in quoted_trades:
        response = hf.show(trade)
        hf_responses.append(response)
    return hf_responses

Run the cell below to check the output for the responses from the Hedge Fund. 

##### In the output below you should see many HfResponse objects with their details.

In [88]:
hf_responses = get_hf_responses(quoted_trades)[:10]

In [89]:
print(hf_responses)

[HfResponse(ticker=FB, trade_volume=3010, trade_price=192.4956, hf_action=sell, ref_price=194.44, bid_price=192.4956, offer_price=196.3844, date=0), HfResponse(ticker=AAPL, trade_volume=6929, trade_price=73.88369999999999, hf_action=sell, ref_price=74.63, bid_price=73.88369999999999, offer_price=75.3763, date=10000), HfResponse(ticker=FB, trade_volume=7015, trade_price=197.7075, hf_action=buy, ref_price=195.75, bid_price=193.7925, offer_price=197.7075, date=30000), HfResponse(ticker=FB, trade_volume=4910, trade_price=195.233, hf_action=buy, ref_price=193.3, bid_price=191.36700000000002, offer_price=195.233, date=70000), HfResponse(ticker=AAPL, trade_volume=5783, trade_price=73.8613, hf_action=buy, ref_price=73.13, bid_price=72.39869999999999, offer_price=73.8613, date=105000), HfResponse(ticker=AAPL, trade_volume=3280, trade_price=74.5683, hf_action=buy, ref_price=73.83, bid_price=73.0917, offer_price=74.5683, date=110000), HfResponse(ticker=FB, trade_volume=2465, trade_price=187.61759

##### The cell below is used to grade the work after the event. You do not need to do anything here.

In [90]:
# GRADING CELL

## e) Begin logging your actions with the Market Maker class and functions.

### Initialise the Market Maker Object

To be able to use the Market Maker in this simulation we run the code in the cell below to create the object. The Market Maker object contains the following properities.

#### Market Maker object contains:
* current_positions: Dictionary of current_position objects - {ticker: String, current_position: Object}
  * current_position: Custom Object
    * ticker: String
    * position_volume: Integer
    * open_price: Float
    * date: Integer
* quoted_trades: List of quoted_trade objects - [quoted_trade, quoted_trade, …]
  * quoted_trade: Custom Object
* completed_trades: List of completed_trade objects - [completed_trade, completed_trade, ...]
  * completed_trade: Custom Object
* ETF_positions: List of completed_trade objects - [completed_trade, completed_trade, ...]
  * completed_trade: Custom Object


In [91]:
VimeoVideo('744734268?h=47a777836e')

In [92]:
mm = MarketMaker.mm()

 As a Market Maker, it is imperative to keep a log of the trades quoted and clients' responses. As a result, interact with the MM object to store the quoted_trades lists into the logs.

#### Add a trade to Market Makers history
```python
mm.add_quoted_trade(QuotedTrade)
```
***Parameters:***
* QuotedTrade: Custom Object

***Returns:***
* String: Indicating a successfully added trade or failed added trade.

In [94]:
for quote in quoted_trades:
    mm.add_quoted_trade(QuotedTrade)

Run the cell below to check the output for market maker quoted_trades. 

##### In the output below you should see many QuotedTrade objects with their details.

In [95]:
print(mm.quoted_trades)

[<class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>, <class '__main__.QuotedTrade'>]


### Create another object to be used during this event following a similar structure to the QuotedTrade class.

**Create a CompletedTrade object**

This object will be used to log successful trades to the market maker object. The object should contain the following attributes:
* ticker: String
* trade_volume: Integer
* trade_price: Float
* mm_action: String
* ref_price: Float
* bid_price: Float
* offer_price: Float
* date: Integer

In [96]:
class CompletedTrade:
    def __init__(self, ticker, trade_volume, trade_price, mm_action, ref_price, bid_price, offer_price, date):
        self.ticker = ticker
        self.trade_volume = trade_volume
        self.trade_price = trade_price
        self.mm_action = mm_action
        self.ref_price = ref_price
        self.bid_price = bid_price
        self.offer_price = offer_price
        self.date = date

### Interact with the MM object to store the CompletedTrade objects into the logs.

#### Update the Market Makers current positions.
```python
mm.add_trade(CompletedTrade)
```
***Parameters:***
* CompletedTrade: Custom Object

***Returns:***
* String: Indicating a successfully added trade or failed added trade.

#### Throughout this simulation the hf_action has the possibility to be the following: "buy", "sell" and "refuse"

In [97]:
def add_completed_trades_to_mm(hf_responses):
    for response in hf_responses:
        if response.hf_action == "refuse":
            continue  # Skip to the next response if it's "refuse"

        # Only create CompletedTrade if hf_action is not "refuse"
        mm_action = "sell" if response.hf_action == "buy" else "buy"
        completed_trade = CompletedTrade(
            response.ticker,
            response.trade_volume,
            response.trade_price,
            mm_action,
            response.ref_price,
            response.bid_price,
            response.offer_price,
            response.date,
        )
        mm.add_trade(completed_trade)

In [98]:
add_completed_trades_to_mm(hf_responses)

Run the cell below to check the output for market maker quoted_trades. 

##### In the output below you should see many CompletedTrade objects with their details.

In [99]:
print(mm.completed_trades)

[<__main__.CompletedTrade object at 0x7fea1139d480>, <__main__.CompletedTrade object at 0x7fea1139e8c0>, <__main__.CompletedTrade object at 0x7fea1139e590>, <__main__.CompletedTrade object at 0x7fea1139fa00>, <__main__.CompletedTrade object at 0x7fea1139d990>, <__main__.CompletedTrade object at 0x7fea1139d240>, <__main__.CompletedTrade object at 0x7fea1139f910>, <__main__.CompletedTrade object at 0x7fea1139fa90>, <__main__.CompletedTrade object at 0x7fea1139f490>, <__main__.CompletedTrade object at 0x7fea1139cb20>]


##### The cell below is used to grade the work after the event. You do not need to do anything here.

In [ ]:
# GRADING CELL

### f) Create a graphical output for all the AAPL quoted trades.

The graph should be created using the matplotlib library. It should show all the quoted trades completed for the ticker AAPL and visualise the *Bid, Offer and Reference Price* against *Date*.

#### Initialise the plot with the <python>fig, axes = plt.subplots() </python> function. 
#### Populate the lists given, and plot them using <python>axes.plot()</python>.

In [ ]:
VimeoVideo('744734758?h=5d07d78c62')

In [ ]:
# Initalises the lists to store all the data
bid_data = []
offer_data = []
quote_dates = []

for trade in mm.completed_trades: # Iterate through the completed trades to find the AAPL trades and store the data.
    # Append the bid_data and offer_data and quote_dates.
    # Insert your answer code here

##### In the print statements below you should be able to see equal length lists for each list.

In [ ]:
print("bid_data:", bid_data)
print("offer_data:", offer_data)
print("quote_dates:", quote_dates)

The quote_dates, bid_data and offer_data are now assigned to the corresponding list. 

#### Next find the reference prices for AAPL as we have gaps in our dates.

In [ ]:
# Replace the ? in this task with your answer.

ref_data = []
ref_dates = []

for price in prices: # Iterate through the quotes to find each AAPL ticker and matching price data.
    # Add filtering to ensure we find AAPL trades and all trades upto the last quote_dates.
    # Append the ref_data and ref_dates
    # Insert your answer code here

##### In the print statements below you should be able to see equal length lists for each list. The ref_dates should go untill the last quote_dates

In [ ]:
print("ref_data:", ref_data)
print("ref_dates:", ref_dates)

#####  Finally we must plot each of the axes.

In [ ]:
axes = plt.subplot() # Creates the Axis.

axes.plot(quote_dates, bid_data) # Plot the X and Y axis using the quote_dates and bid_data.

# Plot the refData and offerData next
# Insert your answer code here

##### The cell below is used to grade the work after the event. You do not need to do anything here.

In [ ]:
# GRADING CELL

![](https://algo-assets.amplifyme.com/quant/challenge2.png)

# Quant Simulation Challenge 2 - Skewed Price Making

In this challenge you must work independently to complete several functions which will now skew your prices based on the position of risk at any given moment of time. This will help show you how to optimize the code.

The goal of this challenge is to automate the market-making process, using object-orientated programming to automatically create a bid-offer spread around the reference price. 

You're expected to provide a 2% non-skewed bid and offer for each trade with no risk.

You're expected to provide a 1% non-skewed bid or offer and a 7% skewed bid or offer depending on your risk.

Good luck!

In [ ]:
VimeoVideo('744735162?h=5b1f522aa1')

In [ ]:
mm = MarketMaker.mm()


### a) Using the code learned in challenge 1 apply an algorithm to all price requests and skew based on risk.


The functions below have been created in 2 steps to allow you to manage the trade process faster.


#### Remember to log any *quoted_trades* and *completed_trades* to your MM object or risk penalties for poor risk management.

#### Function 1 - `def calculate_spread(price):` 
This first function has been mostly done for you, just calculate the bid and offer price. For this function, the *balance* is accessed to determine the firm's risk at any given moment. This information is then used to decide if you're Axed Long, Axed Short or Neutral in your position. This is then reflected in your **bid** and **offer** prices.

In [ ]:
# Replace the ? in this task with your answer.

def calculate_spread(quote):
    volume = mm.current_positions[quote[0][0]].position_volume
    if volume > 0:
        bid_price = ? # Give the bid spread
        offer_price = ? # Give the offer spread
    # Complete all other scenarios for the volume.
    # Create a QuotedTrade to be returned from this function and log it.
    # Insert your answer code here
    return trade

#### Function 2 - `def handle_response(trade):` 
The **QuotedTrade** object is shown using the HF's function, to get the HF's response. You need to handle this response, in order to create your **CompletedTrade** object correctly. Also, log your CompletedTrade object using: `mm.add_trade(CompletedTrade)` .

In [ ]:
def handle_response(trade):
    # Add the logic here for handling the response action.
    # Create and Store the CompletedTrade into your logs.
    # Insert your answer code here

### Run the cell below to call your functions

In [ ]:
for request in price_requests:
    for price in prices:
        if price[0] == request[0] and price[1] == request[1]:
            quote = calculate_spread( (request, price[2]) )
            response = hf.show(quote)
            trade = handle_response(response)

In [ ]:
# GRADING CELL

### b) Create graphical outputs for all the tickers you quoted_trades

#### Similar to the graph created in Chalenge One however it should be done for all the different tickers dealt for the HF client.

In [ ]:
# Insert your answer code here